<a href="https://colab.research.google.com/github/huzaifa003/NewsMachineLearning/blob/main/Fine_Tuning_MT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow transformers

In [ ]:
import pandas as pd

# Load the dataset
file_path = 'news_20k.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset and its structure
data.head(), data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   20000 non-null  int64 
 1   Index        20000 non-null  int64 
 2   Headline     20000 non-null  object
 3   News Text    20000 non-null  object
 4   Category     20000 non-null  object
 5   Date         19376 non-null  object
 6   URL          20000 non-null  object
 7   Source       20000 non-null  object
 8   News length  20000 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 1.4+ MB


(   Unnamed: 0    Index                                           Headline  \
 0      276152   276152  لاہورہائیکورٹ نے محکمہ مال کو زرعی مدن ٹیکس کی...   
 1      257971   257971  زرعی شعبہ میں تحقیقی سرگرمیوں کے فروغ کے لئی12...   
 2      217973   217973                        ایشین سٹاک مارکیٹس میں مندی   
 3      297403   297403  کراچی پاکستان پیٹر ولئیم کیعمومی شئیرکیپٹل پر7...   
 4     1002075  1002075  اسٹاک مارکیٹ انڈیکس میں86 پوائنٹس کمی سرمایہ م...   
 
                                            News Text              Category  \
 0  لاہوراردو پوائنٹ اخبارتازہ ترین لائن16 مئی2016...  Business & Economics   
 1  اسلام اباد اردو پوائنٹ اخبارتازہ ترین اے پی پی...  Business & Economics   
 2  ہانگ کانگ اردو پوائنٹ اخبارتازہ ترین اے پی پی ...  Business & Economics   
 3  کراچی اردو پوائنٹ اخبار تازہ ترین24 اکتوبر2014...  Business & Economics   
 4  کراچیکامرس رپورٹر پاکستان اسٹاک ایکسچینج پی ای...  Business & Economics   
 
              Date                                    

In [ ]:
import re

def clean_text(text):
    # Replace new lines and returns with space
    text = re.sub(r'\s+', ' ', text)
    # Remove any leading or trailing whitespace
    text = text.strip()
    return text

# Clean the 'Headline' and 'News Text' columns
data['Headline'] = data['Headline'].apply(clean_text)
data['News Text'] = data['News Text'].apply(clean_text)

# Display the cleaned text samples
data[['Headline', 'News Text']].head()


,Headline,News Text
0,لاہورہائیکورٹ نے محکمہ مال کو زرعی مدن ٹیکس کی...,لاہوراردو پوائنٹ اخبارتازہ ترین لائن16 مئی2016...
1,زرعی شعبہ میں تحقیقی سرگرمیوں کے فروغ کے لئی12...,اسلام اباد اردو پوائنٹ اخبارتازہ ترین اے پی پی...
2,ایشین سٹاک مارکیٹس میں مندی,ہانگ کانگ اردو پوائنٹ اخبارتازہ ترین اے پی پی ...
3,کراچی پاکستان پیٹر ولئیم کیعمومی شئیرکیپٹل پر7...,کراچی اردو پوائنٹ اخبار تازہ ترین24 اکتوبر2014...
4,اسٹاک مارکیٹ انڈیکس میں86 پوائنٹس کمی سرمایہ م...,کراچیکامرس رپورٹر پاکستان اسٹاک ایکسچینج پی ای...


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets (80% train, 20% validate)
train_data, validate_data = train_test_split(data, test_size=0.2, random_state=42)

# Display the size of the training and validation sets
train_data.shape, validate_data.shape


((16000, 9), (4000, 9))

In [ ]:
# Import necessary modules from transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
import tensorflow as tf

# Load the T5 model and tokenizer
model_name = 't5-small'  # We use 't5-small' for demonstration due to resource constraints
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Check if TensorFlow is properly configured to use GPU if available
tf.config.list_physical_devices('GPU')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
inputs = tokenizer(["summarize: " + text for text in train_data['News Text'].tolist()], return_tensors="tf", max_length=512, truncation=True, padding="max_length")
targets = tokenizer(train_data['Headline'].tolist(), return_tensors="tf", max_length=128, truncation=True, padding="max_length")


In [ ]:
# # Create tf.data.Dataset
# dataset = tf.data.Dataset.from_tensor_slices(({
#     "input_ids": inputs['input_ids'],
#     "attention_mask": inputs['attention_mask'],
#     "decoder_input_ids": targets['input_ids']
# }, targets['input_ids']))

# dataset = dataset.shuffle(buffer_size=10000).batch(8)  # Shuffle and batch the data

In [ ]:
model = None

In [ ]:
# from transformers import TFT5ForConditionalGeneration, T5Tokenizer
# import tensorflow as tf

# # Load tokenizer and model
# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# model = TFT5ForConditionalGeneration.from_pretrained('t5-small')

# # Compile the model
# model.compile(optimizer='adam', loss=model.compute_loss)  # Using the built-in compute_loss of T5


In [ ]:
# history = model.fit(dataset, epochs=3)  # Train for 3 epochs

In [ ]:
# print("Check input ids:", inputs['input_ids'])
# print("Check attention masks:", inputs['attention_mask'])
# print("Check decoder input ids:", targets['input_ids'])


In [ ]:
from transformers import TFT5ForConditionalGeneration, T5Tokenizer
import tensorflow as tf

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = TFT5ForConditionalGeneration.from_pretrained('t5-small')

# Assume inputs and targets have already been tokenized and are available as TensorFlow tensors
batch_size = 8  # Define the batch size that fits your GPU memory

# Dataset creation with batching
dataset = tf.data.Dataset.from_tensor_slices((inputs.input_ids, targets.input_ids))
dataset = dataset.batch(batch_size)

# Training loop with batches
optimizer = tf.optimizers.Adam()

for input_ids, labels in dataset:
    with tf.GradientTape() as tape:
        # Forward pass in batches
        outputs = model(input_ids, labels=labels)
        loss = outputs.loss

    # Calculate gradients
    gradients = tape.gradient(loss, model.trainable_variables)

    # Apply gradients (backpropagation)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print(f"Batch loss: {loss.numpy()}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Batch loss: [2.1583579]
Batch loss: [0.66967374]
Batch loss: [0.18649133]
Batch loss: [0.18675628]
Batch loss: [0.11676572]
Batch loss: [0.13307138]
Batch loss: [0.07510875]
Batch loss: [0.08288907]
Batch loss: [0.07523438]
Batch loss: [0.1037224]
Batch loss: [0.16516715]
Batch loss: [0.09767098]
Batch loss: [0.0962348]
Batch loss: [0.06544062]
Batch loss: [0.06505723]
Batch loss: [0.05200386]
Batch loss: [0.05547721]
Batch loss: [0.05773696]
Batch loss: [0.04418668]
Batch loss: [0.05151638]
Batch loss: [0.04138219]
Batch loss: [0.07094979]
Batch loss: [0.06325611]
Batch loss: [0.06036944]
Batch loss: [0.04385406]
Batch loss: [0.05161969]
Batch loss: [0.06535547]
Batch loss: [0.03402868]
Batch loss: [0.02926998]
Batch loss: [0.02771286]
Batch loss: [0.07158709]
Batch loss: [0.03205939]
Batch loss: [0.04232605]
Batch loss: [0.03131637]
Batch loss: [0.06301593]
Batch loss: [0.05708649]
Batch loss: [0.03045156]
Batch loss: [0.05481459]
Batch loss: [0.04749392]
Batch loss: [0.02656266]
Bat

In [ ]:
model.save_pretrained('/content/drive/MyDrive/Urdu_NLP/t5_model_fine_tuned_news_headline')

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/Urdu_NLP/t5_model_fine_tuned_news_headline_tokenizer')

('/content/drive/MyDrive/Urdu_NLP/t5_model_fine_tuned_news_headline_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Urdu_NLP/t5_model_fine_tuned_news_headline_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Urdu_NLP/t5_model_fine_tuned_news_headline_tokenizer/spiece.model',
 '/content/drive/MyDrive/Urdu_NLP/t5_model_fine_tuned_news_headline_tokenizer/added_tokens.json')

In [ ]:
# Example news article in Urdu
news_article_urdu = "پاکستان میں تعلیمی اصلاحات کا آغاز ہو چکا ہے۔ حکومت نے ملک بھر میں جدید تعلیمی پالیسیوں کو نافذ کرنے کا فیصلہ کیا ہے جس کا مقصد طلبہ کی معیاری تعلیم کو یقینی بنانا ہے۔ اس نئی پالیسی کے تحت، سکولوں میں جدید تعلیمی مواد اور طریقہ تدریس کو متعارف کروایا جائے گا، جو کہ طلبہ کے لیے جدید دور کے تقاضوں کو پورا کرنے میں مددگار ثابت ہوگا۔"

# Prepare the text with the same format used during training
input_text_urdu = "summarize: " + news_article_urdu

# Tokenize the text
input_ids_urdu = tokenizer(input_text_urdu, return_tensors="tf", max_length=512, truncation=True).input_ids


In [ ]:
# Generate the headline
generated_ids = model.generate(
    input_ids_urdu,
    max_length=50,    # Maximum length of the output headline
    num_beams=5,      # Use beam search with 5 beams
    no_repeat_ngram_size=2,  # Prevent repeating n-grams
    early_stopping=True      # Stop as soon as num_beams sentences are fully generated
)

# Decode the generated ids to get the text
headline = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated Headline:", headline)


Generated Headline: 2019


In [ ]:
print("Check tokenized text:", tokenizer.decode(input_ids_urdu[0]))


Check tokenized text: summarize: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk></s>


In [4]:
news_article_urdu = "پاکستان میں تعلیمی اصلاحات کا آغاز ہو چکا ہے۔ حکومت نے ملک بھر میں جدید تعلیمی پالیسیوں کو نافذ کرنے کا فیصلہ کیا ہے جس کا مقصد طلبہ کی معیاری تعلیم کو یقینی بنانا ہے۔ اس نئی پالیسی کے تحت، سکولوں میں جدید تعلیمی مواد اور طریقہ تدریس کو متعارف کروایا جائے گا، جو کہ طلبہ کے لیے جدید دور کے تقاضوں کو پورا کرنے میں مددگار ثابت ہوگا۔"
from transformers import MT5ForConditionalGeneration, T5Tokenizer

# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")

# Tokenize the Urdu text
input_text_urdu = "summarize: " + news_article_urdu
input_ids_urdu = tokenizer(input_text_urdu, return_tensors="tf", max_length=512, truncation=True).input_ids

import tensorflow as tf



/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


TypeError: MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False

In [5]:
article = "UN Offizier sagt, dass weiter verhandelt werden muss in Syrien."
summary = "Weiter Verhandlung in Syrien."
inputs = tokenizer(article, text_target=summary, return_tensors="pt")

outputs = model(**inputs)
loss = outputs.loss

In [9]:
outputs = model.generate(inputs['input_ids'])
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<extra_id_0> - UN


In [3]:
outputs

Seq2SeqLMOutput(loss=tensor(16.7407, grad_fn=<NllLossBackward0>), logits=tensor([[[-59.3139, -31.2654, -59.4377,  ..., -61.5389, -62.3253, -60.6338],
         [-52.7880, -18.6159, -52.5216,  ..., -52.9742, -53.9897, -53.4395],
         [-65.7384,  -4.6215, -66.5576,  ..., -69.3745, -69.0690, -67.3726],
         ...,
         [-57.3241, -10.8157, -57.1157,  ..., -58.2630, -59.4203, -57.9627],
         [-59.6123, -10.2896, -59.5021,  ..., -60.7650, -61.7202, -60.7405],
         [-46.2160, -16.9575, -46.4631,  ..., -47.0272, -47.8878, -47.1938]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[ 0.2816,  0.9202,  0.5680,  ...,  1.7564, -0.4593,  0.7998],
          [ 1.8032, -0.7471,  1.0907,  ..., -0.4927,  0.3512, -0.0191],
          [ 0.1850, -0.6111, -0.5535,  ...,  0.8196, -0.6117, -0.2042],
          ...,
          [ 0.0542, -0.4713, -0.0212,  ..., -0.0560,  0.5658,  0.0633],
          [ 0.8814,  0.2737,  0.0119,  ..., -0.4505,  0.1547, -0.0056],
          [ 0.352

In [2]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

def generate_headline(text, model_name="google/mt5-base"):
    # Load model and tokenizer
    model = MT5ForConditionalGeneration.from_pretrained(model_name)
    tokenizer = MT5Tokenizer.from_pretrained(model_name)

    # Encode the text with a more specific prompt for headline generation
    input_ids = tokenizer.encode("create headline: " + text, return_tensors="pt", max_length=512, truncation=True)

    # Generate output with more exploratory parameters
    summary_ids = model.generate(input_ids, num_beams=10, no_repeat_ngram_size=3, max_length=60, early_stopping=False)

    # Decode and print the summary
    headline = tokenizer.decode(summary_ids[0], skip_special_tokens=True).replace("<extra_id_0>", "")
    return headline

# Example Urdu text
urdu_text = "یہ ایک مثالی خبر کا متن ہے جس کا سرخی تیار کرنی ہے۔"
headline = generate_headline(urdu_text)
print("Generated Headline:", headline)


Generated Headline:  یہ ایک مثال


In [5]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [6]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [1]:
import pandas as pd

# Load the dataset to examine its structure
file_path = 'news_20k.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
data.head()


,Unnamed: 0,Index,Headline,News Text,Category,Date,URL,Source,News length
0,276152,276152,لاہورہائیکورٹ نے محکمہ مال کو زرعی مدن ٹیکس کی...,لاہوراردو پوائنٹ اخبارتازہ ترین لائن16 مئی2016...,Business & Economics,NaN,https://www.urdupoint.com/business/news-detail...,Urdu Point News,542
1,257971,257971,زرعی شعبہ میں تحقیقی سرگرمیوں کے فروغ کے لئی12...,اسلام اباد اردو پوائنٹ اخبارتازہ ترین اے پی پی...,Business & Economics,10 مارچ2017ء,https://www.urdupoint.com/business/news-detail...,Urdu Point News,698
2,217973,217973,ایشین سٹاک مارکیٹس میں مندی,ہانگ کانگ اردو پوائنٹ اخبارتازہ ترین اے پی پی ...,Business & Economics,16 ستمبر2019ء,https://www.urdupoint.com/business/news-detail...,Urdu Point News,310
3,297403,297403,کراچی پاکستان پیٹر ولئیم کیعمومی شئیرکیپٹل پر7...,کراچی اردو پوائنٹ اخبار تازہ ترین24 اکتوبر2014...,Business & Economics,NaN,https://www.urdupoint.com/business/news-detail...,Urdu Point News,1663
4,1002075,1002075,اسٹاک مارکیٹ انڈیکس میں86 پوائنٹس کمی سرمایہ م...,کراچیکامرس رپورٹر پاکستان اسٹاک ایکسچینج پی ای...,Business & Economics,"Jun 30, 2018",https://www.nawaiwaqt.com.pk/30-Jun-2018/855776,Nawai Waqt News,1210


In [2]:
# Drop rows where 'Headline' or 'News Text' might be missing
cleaned_data = data.dropna(subset=['Headline', 'News Text'])

# Let's also remove any potential duplicates
cleaned_data = cleaned_data.drop_duplicates(subset=['News Text'])

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(cleaned_data, test_size=0.1, random_state=42)

# Check the size of the splits and a few entries to ensure everything is in order
(train_data.shape, val_data.shape, train_data.head())


((18000, 9),
 (2000, 9),
        Unnamed: 0    Index                                           Headline  \
 16270      889314   889314  برسبین ٹیسٹ میں سمتھ کی ذمہ دارانہ بیٹنگ سٹریل...   
 1383       271258   271258         سویابین کی کاشت بروقت مکمل کریںمحکمہ زراعت   
 3472      1028302  1028302  کوئٹہ چمن روڈ کی تعمیر نو کیلئے امریکہ 45 ملین...   
 19900      173692   173692  ٹیم کا مورال بلند کرنے کیلئے پاکستان کو زیادہ ...   
 2150       111495   111495      پی ائی اے میں گھوسٹ ملازمین کے خلاف گھیرا تنگ   
 
                                                News Text  \
 16270  برسبینسپورٹس ڈیسک ایشز سیریز کے پہلے ٹیسٹ کے ت...   
 1383   فیصل باد2 ستمبراردو پوائنٹ اخبارتازہ ترین اے پ...   
 3472   اسلام باد اے پی پی نیشنل ہائی وے اتھارٹی کے چی...   
 19900  سڑیلیا میں دنیا کی بہترین ڈومیسٹک کرکٹ کھیلی ج...   
 2150   مہینے میں3بار تاخیر پر ایک دن کی چھٹی تصورکی ج...   
 
                    Category                 Date  \
 16270                Sports         Nov 26, 2017   
 1383  

In [3]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from datasets import Dataset

# Initialize tokenizer
model_name = "google/mt5-base"
tokenizer = MT5Tokenizer.from_pretrained(model_name)

# Function to encode the data
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["News Text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["Headline"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Create datasets objects
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Apply the preprocess function
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Check the first example
train_dataset[0]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'Unnamed: 0': 889314,
 'Index': 889314,
 'Headline': 'برسبین ٹیسٹ میں سمتھ کی ذمہ دارانہ بیٹنگ سٹریلیا کی پوزیشن مستحکم',
 'News Text': 'برسبینسپورٹس ڈیسک ایشز سیریز کے پہلے ٹیسٹ کے تیسرے دن سٹریلوی ٹیم پہلی اننگز میں 328 رنز بنا کر وٹ ہوگئیتیسرے دن سٹریلیا نے 165 رنز چار کھلاڑی وٹ سے اننگز گے بڑھائی تو مچل مارش51 رنز بناکر وٹ ہوگئے تاہم کپتان سٹیون سمتھ 141 رنز پر ناقابل شکست رہے اور سکور328 رنز تک پہنچا دیاسٹورٹ براڈ 3وکٹیں لے کر نمایاں رہے26 رنز کے خسارے سے انگلینڈ نے اپنی دوسری اننگز میں دن کے اختتام تک دو وکٹ کھو کر 33 رنز بنالئے ہیںسٹون مین 19 اور جو روٹ رنز بناکر کریز پر موجود ہیںانگلش ٹیم پہلی اننگز میں 302رنز بناکر ہوگئی تھی ناگ پورٹیسٹ بھارت نے 2وکٹوں پر 312رنز بناکر پوزیشن مستحکم کر لی ناگ پورسپورٹس ڈیسک بھارت نے سیریز کے دوسرے ٹیسٹ میچ میںدو وکٹوں پر312رنز بناکر 107رنز کی برتری حاصل کر لی مرلی وجے 129رنز بناکر وئٹ ہوئے پچار ا121اور ویرات کوہلی54رنز بناکر کھیل رہے ہیں سری لنکا نے پہلی اننگز میں 205رنز بنائے تھے',
 'Category': 'Sports',
 'Date': 'Nov 26, 2017',
 'URL': 'http

In [ ]:
# Import necessary libraries
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from datasets import Dataset, load_metric

# Load tokenizer and model
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-base")
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-base")

# Data collator used for padding and creating batches
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,

)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start training
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
